In [228]:
import numpy as np
import struct
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline
import tqdm

## Read dynamic list-mode CASToR datafile

In [229]:
datafile='poumon_pat6_list-mode_0_tof_ref0_compr_1_10.cdf'

In [244]:
# Structure of datafile for 32 bit list-mode ( including TOF )
struct.calcsize('IfffffII')


# Structure of compressed datafile for 24 bit list-mode ( including TOF )
# uint32_t : time tag
# float: Normalization + Attenuation
# float: Scatter + randoms
# float: TOF arrival time diff
# uint32_t: CrystalID1
# uint32_t: CrystalID2
struct.calcsize('IfffII')

24

In [245]:
# Get last time tag
with open(datafile, mode='rb') as file: # b is important -> binary
    file.seek(os.path.getsize(datafile)-24)
    # Read last phrase
    datafilePack = file.read(24)
    # read its time tag , as the last time tag
    lastTimeTag = int(struct.unpack("I",datafilePack[:4])[0])

In [246]:
lastTimeTag

3599999

In [247]:
# Unpack file in 32 bits and read the first 4 bits (time tags) as an unsigned int
counter=0
with open(datafile, mode='rb') as file: # b is important -> binary
    #file.seek(os.path.getsize(datafile)-32*2)
    datafilePack = file.read(24)
    counter+=1
    # Get time tag 
    timetag = int(struct.unpack("I",datafilePack[:4])[0])
    while datafilePack:
        if (False):
            print(counter)
            break;
        datafilePack = file.read(24)
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        counter+=1

error: unpack requires a buffer of 4 bytes

In [248]:
counter

417017835

In [250]:
os.path.getsize(datafile)/24

417017835.0

In [249]:
timetag

3599999

## Define Framing for the study ( on study refference time == injection time )

In [20]:
frm="10, 20, 30, 40, 50, 60, 70, 80,  \
90, 100, 110, 120, 130, 150, 170, 190, 210, 230,  \
250, 270, 290, 310, 330, 350, 370, 430, 490, 550, 610, 730, 850, 970, \
1090, 1210, 1510, 1810, 2110, 2410, 2710, 3010, 3310:300"

In [21]:
frames_phrases=frm.split(",")
FrameStart=np.zeros([len(frames_phrases)])
FrameDuration=np.zeros([len(frames_phrases)])

fr=0
for phrase in frames_phrases:
    #print(phrase)
    if (phrase.find(":") != -1) :
        FrameStart[fr]=float(phrase.split(":")[0])
        FrameDuration[fr]=float(phrase.split(":")[1])
    else:
        FrameStart[fr]=float(phrase)
    fr+=1

# Loop over farmes duration and replace zeros with time until next frame
for idx,val in enumerate(FrameDuration):
    if val==0:
        FrameDuration[idx]=FrameStart[idx+1]-FrameStart[idx]

## Read respiratory triggers time tags

In [155]:
resp_triggers=pd.read_csv('data/LIST0000.resp')
resp_triggers.columns = ['trigger_time_tag']

In [156]:
resp_triggers.diff().mean()

trigger_time_tag    3416.174766
dtype: float64

In [157]:
resp_triggers = np.array(resp_triggers.trigger_time_tag.tolist())
resp_triggers_diff = np.diff(resp_triggers)

In [158]:
# Statistics of resp tags
print("Number of respiratory tags: %d"%resp_triggers.shape[0])
print("Mean respiratory rate: %f ms"%np.diff(resp_triggers).mean())
accepted_count=((np.diff(resp_triggers)<3000*1.3) & (np.diff(resp_triggers)>3000*0.7)).sum()
print("Number of cycles to include: %d "%accepted_count)
print("Rejection fraction: {0:.4f}%".format((1-accepted_count/resp_triggers.shape[0])*100))

Number of respiratory tags: 1071
Mean respiratory rate: 3416.174766 ms
Number of cycles to include: 869 
Rejection fraction: 18.8609%


## Create time-tag info array for gates phase, gate index and rejected triger phases

### In this step we create an information matrix for each time-tag within the list mode file.
In each time tag we have the following information:
#0: time-tag
#1: respiratory cycle index
#2: Accepted or rejected  based on respiratory cycle duration ( within 30% of 3000ms ) 
#3: Use for Q.Static ?  (Yes=1 or No=0)
#4: Q.Freeze (Gate) index number

In [172]:
# Fake lastTimeTag for development 
lastTimeTag = 3656802+1000

In [173]:
InfoMatrix = np.zeros([lastTimeTag,5])

In [174]:
InfoMatrix[:,0] = np.arange(lastTimeTag)

In [175]:
resp_triggers[-1]

3656802

In [223]:
# Set respiratory cycle index and check respiratory cycle duration
current_resp_cycle=0
# Reject all tags before the first respiratory trigger and after the last 
for tag in range(resp_triggers[0]):
    InfoMatrix[tag,1]=0
    InfoMatrix[tag,2]=0

for tag in range(resp_triggers[-1],lastTimeTag):
    InfoMatrix[tag,1]=0
    InfoMatrix[tag,2]=0

for tag in tqdm.tqdm(range(resp_triggers[0],resp_triggers[-1])):
    # it tag is greater or equal to the following resp trigger, move one cycle up
    if tag>=resp_triggers[current_resp_cycle+1]:
        current_resp_cycle+=1;
    # Set the current_resp_cycle index
    InfoMatrix[tag,1]=current_resp_cycle
    if (3000*0.7<resp_triggers_diff[current_resp_cycle]<3000*1.3):
        InfoMatrix[tag,2]=1
    
    # Trigger is placed 30% forward from the peak ( by GE )
    # we keep the first 50% of the data in the resp cycle for Q.Static data
    if (tag<resp_triggers[current_resp_cycle]+resp_triggers_diff[current_resp_cycle]/2):
        InfoMatrix[tag,3]=1
      
    # Trigger is placed 30% forward from the peak ( by GE )
    # we splitt the data in the resp cycle in 5 parts/gates for Q.Static data    
    for gate in range(1,6):
        gate_start=resp_triggers[current_resp_cycle]+(resp_triggers_diff[current_resp_cycle]/5.)*(gate-1)
        gate_end=resp_triggers[current_resp_cycle]+(resp_triggers_diff[current_resp_cycle]/5.)*(gate)
        if (gate_start<=tag<gate_end):
            InfoMatrix[tag,4]=gate


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3655307/3655307 [03:22<00:00, 18027.56it/s]


In [226]:
#InfoMatrix[1495:1555]
InfoMatrix[1555:4915]

array([[1.555e+03, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00],
       [1.556e+03, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00],
       [1.557e+03, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00],
       ...,
       [4.912e+03, 1.000e+00, 1.000e+00, 0.000e+00, 5.000e+00],
       [4.913e+03, 1.000e+00, 1.000e+00, 0.000e+00, 5.000e+00],
       [4.914e+03, 1.000e+00, 1.000e+00, 0.000e+00, 5.000e+00]])

## Process the list-mode file and create Q.Static Q.Freeze list files (with frames)